In [1]:
import urllib3.request
from bs4 import BeautifulSoup
import json
import pandas as pd
urllib3.disable_warnings()

##links
url1 = 'https://quotes-gw.webullbroker.com/api/search/pc/tickers?keyword=evc&pageIndex=1&pageSize=20'
url2 = 'https://quotes-gw.webullfintech.com/api/quotes/chip/query?tickerId=913255272&startDate=2020-09-01&endDate=2020-10-31'



In [2]:
##real code starts here
f = open("symbolsInput-2.txt", 'r')
symbols_list = f.read().split("\n")
http = urllib3.PoolManager()
tickerIDs = []
symbol_tck = []

for sl in symbols_list:
    html = http.request('GET', 'https://quotes-gw.webullbroker.com/api/search/pc/tickers?keyword='+str(sl)+'&pageIndex=1&pageSize=20')
    soup = BeautifulSoup(html.data, features="lxml").find('p').get_text()
    dt = json.loads(soup)
    df = pd.DataFrame(dt['data'])
    tickerIDs.append(list(df['tickerId']))

In [3]:
df123 = pd.DataFrame({"symbol" : symbols_list, "ti" : tickerIDs})

In [4]:
df123

,symbol,ti
0,KGJI,[913253342]
1,TAT,"[913303902, 913323551, 950102446, 925309751, 9..."
2,OBLG,"[913303685, 913317116, 925220949, 925274047, 9..."
3,SNDE,[916040700]
4,SPI,"[913256084, 925283514, 913255655, 913324358, 9..."
5,NTEC,"[916040768, 925287914, 925289346, 925289348, 9..."
6,RWLK,[913255915]
7,GMO,"[925163614, 913303810, 925302019, 950095874, 9..."
8,HMHC,[913255799]
9,AMPY,"[913435351, 950099255, 925339023]"


In [5]:
df456 = df123.explode('ti')

In [6]:
row = []

for ticker in df456['ti']:
    html = http.request('GET', 'https://quotes-gw.webullfintech.com/api/quotes/chip/query?tickerId='+str(ticker)+'&startDate=2020-09-01&endDate=2020-10-31')
    soup = BeautifulSoup(html.data, features="lxml").find('p').get_text()
    dt2 = json.loads(soup)
    df2 = pd.DataFrame(dt2['data'])
    if not df2.empty:  
        symb = df456[df456['ti'] == ticker]['symbol'].to_string().split()[1]
        row.append({'symbol': symb, 'ticker': ticker, 'avgCost': df2['avgCost'], 'close': df2['close']})   

In [7]:
newdf = pd.DataFrame(row)
newdf_avgcost = newdf.drop(columns=["close"]).explode('avgCost')
newdf_close = newdf.drop(columns=["avgCost"]).explode('close')

In [31]:
newdf_avgcost = newdf_avgcost.combine_first(newdf_close)

In [50]:
unique_count = newdf_avgcost.value_counts('ticker').max()
data = newdf_avgcost.transpose().to_dict('list')

with open('test_file.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([" "])
    writer.writerow(["symbol", "tickerID", (" ", "avgCost", "close", " ") * unique_count])
    for key, value in data.items():
        writer.writerow([" ", key, value, " "])

<ipython-input-50-5ea99e7c89bf>:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  data = newdf_avgcost.transpose().to_dict('list')
